In [2]:
import pandas as pd
import sqlite3
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
df['order_date'] = pd.to_datetime(df['order_date'])

C:\Users\mdjaveedkhan\AppData\Local\Temp\ipykernel_6332\2765250300.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [7]:
gold_city_revenue = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [8]:
df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [9]:
user_spend = df.groupby('user_id')['total_amount'].sum()
count_users = (user_spend > 1000).sum()
count_users

np.int64(2544)

In [10]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"]

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)

df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

C:\Users\mdjaveedkhan\AppData\Local\Temp\ipykernel_6332\1554912688.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [11]:
(
    df[df['membership']=='Gold']
    .groupby('city')['total_amount']
    .mean()
    .sort_values(ascending=False)
)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [12]:
restaurant_counts = df.groupby('cuisine')['restaurant_id'].nunique()
revenue = df.groupby('cuisine')['total_amount'].sum()

pd.concat([restaurant_counts, revenue], axis=1)


,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [13]:
gold_orders = len(df[df['membership']=='Gold'])
total_orders = len(df)

percentage = round((gold_orders/total_orders)*100)
percentage


50

In [18]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered = restaurant_stats.loc[options]
filtered
filtered[filtered['total_orders'] < 20].sort_values('avg_value', ascending=False)


,avg_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [20]:
# create combination column
df['combo'] = df['membership'] + " + " + df['cuisine']

# total revenue per combo
combo_revenue = df.groupby('combo')['total_amount'].sum().sort_values(ascending=False)

combo_revenue

options = [
    "Gold + Indian",
    "Gold + Italian",
    "Regular + Indian",
    "Regular + Chinese"
]

combo_revenue.loc[options].sort_values(ascending=False)


combo
Gold + Italian       1005779.05
Regular + Indian      992100.27
Gold + Indian         979312.31
Regular + Chinese     952790.91
Name: total_amount, dtype: float64

In [17]:
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

## 2 Numeric

In [23]:
#How many total orders were placed by users with Gold membership?
gold_orders = df[df['membership'] == 'Gold'].shape[0]
gold_orders


4987

In [24]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
hyd_revenue = round(
    df[df['city'] == 'Hyderabad']['total_amount'].sum()
)
hyd_revenue


1889367

In [25]:
#How many distinct users placed at least one order?
distinct_users = df['user_id'].nunique()
distinct_users

2883

In [26]:
#What is the average order value (rounded to 2 decimals) for Gold members?
avg_gold_value = round(
    df[df['membership'] == 'Gold']['total_amount'].mean(), 2
)
avg_gold_value


np.float64(797.15)

In [27]:
#How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders = df[df['rating'] >= 4.5].shape[0]
high_rating_orders


3374

In [28]:
#How many orders were placed in the top revenue city among Gold members only?
top_city = (
    df[df['membership']=='Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)
top_city
orders_top_city = df[
    (df['membership']=='Gold') &
    (df['city']==top_city)
].shape[0]

orders_top_city


1337